
# Генераторы и асинхронность.



Что такое генератор? Не каждый хороший питонист сможет правильно ответить на этот вопрос. Под капотом эта конструкция скрывает себе множество неочевидных вещей, в которых мы попробуем разобраться



In [2]:

# В первую очередь генератор - это функция.
# Вместо ключевого слова `return` написан `yield`
# Любая функция с `yield` становится генератором

def gen(s: str):
    for x in s:
        yield x


In [ ]:

# Попробуем использовать наш генератор
gen('hello!')


In [ ]:
# Чёт ничего не получилось :( 
# На самом деле, при вызове, генератор не начинает выполнение функции,
# а создаёт объект `generator object`

g = gen('hello!')

for x in g:
    print(x)


In [ ]:
# На самом деле эта конструкция представима как

g = gen('hello!')

print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))
print(next(g))

# Эксперимент! Вызовем next ещё раз
next(g)


In [3]:
# Отлично, теперь мы знаем, как выполнять генератор до конца


g = gen('hello!')

while True:
    try:
        print(next(g))
    except StopIteration:
        break

h
e
l
l
o
!


Резумируем несколько важных вещей, про которые вы, возможно, уже догадались

 - Генератор - это функция
 - Генератор - это необычная функция
 - Генератор - это функция, которая умеет останавливаться!
 

In [ ]:
# В генераторе может быть несколько инструкций yield

def multigen(x: int):
    x += 1
    yield x
    
    x *= 42

In [4]:

# В генераторе может быть несколько инструкций yield

def multigen(x: int):
    x += 1
    yield x
    
    x *= 42
    yield x
    
    x -= 10
    yield x


g = multigen(1)
print(next(g))

# Заметьте, что тут мы можем написать какой-то другой код
print('Here!')

print(next(g))
print(next(g))


2
Here!
84
74


In [5]:


def bidirectional_gen():
    message = yield
    message2 = yield message.lower()
    yield message2


g = bidirectional_gen()
g.send(None)  # равносильно next(g)

resp = g.send('HELLO!')
print(f"Got response from gen: {resp}")
g.send('World')

Got response from gen: hello!


'World'

In [19]:

def average():
    s = 0
    n = 0
    while True:
        s += yield s / n if n > 0 else 0
        n += 1

g = average()
g.send(None)
print(g.send(10))  # 10
print(g.send(10))  # 10
print(g.send(8))   # 9.333...
print(g.send(8))   # 9


10.0
10.0
9.333333333333334
9.0


In [24]:

def subgen(s: str):
    for x in s:
        try:
            yield x
        except Exception:
            print('Exception here')
            break
        

def delegator(s: str):
    g = subgen(s)
    
#     command = None
#     while True:
#         try:
#             command = yield g.send(command)
#         except Exception as e:
#             g.throw(e)

    yield from g



    
dg = delegator('hello')
print(dg.send(None))  # h
print(dg.send(None))  # e
print(dg.throw(Exception))  # Exception here + StopIteration
print(dg.send(None))


h
e
Exception here
w
o



Поговорим про парадигмы асинхронного программирования <br>
https://ru.wikipedia.org/wiki/Round-robin_(%D0%B0%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC)



### Пишем свою асинхронность!



In [ ]:
import time
import itertools
from inspect import getgeneratorstate


def first_task():
    for i in itertools.count():
        time.sleep(1)  # какие-то вычисления. На самом деле, всё работает совсем по другому.
        print(f'Iteration #{i}')
        yield


def second_task():
    i = 0
    while True:  
        time.sleep(1)
        i = (i + 1) % 5  # каждые 5 секунд
  
        if i == 0:
            print(time.time())
        yield
    

tasks = [first_task(), second_task()]


def run_event_loop(t):
    while True:
        task = t.pop(0)
        task.send(None)
        t.append(task)
        

run_event_loop(tasks)


Iteration #0
Iteration #1
Iteration #2
Iteration #3
Iteration #4
1570536317.1702576
Iteration #5
Iteration #6
Iteration #7
Iteration #8
Iteration #9
1570536327.222872
Iteration #10
Iteration #11
Iteration #12
Iteration #13
Iteration #14
1570536337.269929
Iteration #15
Iteration #16
Iteration #17
Iteration #18
Iteration #19
1570536347.3549685
Iteration #20
Iteration #21
Iteration #22
Iteration #23
Iteration #24
1570536357.4112914
Iteration #25
Iteration #26
Iteration #27
Iteration #28
Iteration #29
1570536367.4768302
Iteration #30
Iteration #31
Iteration #32
Iteration #33
Iteration #34
1570536377.5274851
Iteration #35
Iteration #36
Iteration #37
Iteration #38
Iteration #39
1570536387.6171236
Iteration #40
Iteration #41
Iteration #42
Iteration #43
Iteration #44
1570536397.6688013
Iteration #45
Iteration #46
Iteration #47
Iteration #48
Iteration #49
1570536407.7116883
Iteration #50
Iteration #51
Iteration #52
Iteration #53
Iteration #54
1570536417.7939885
Iteration #55
Iteration #56
Itera

In [ ]:
# ЗАПУСКАЙТЕ В ФАЙЛЕ!
# Это старая асинхронность до версии 3.5
# Тут вы встретите знакомые yield from. Аналогично await в версии 3.5+



import asyncio
import itertools
import time


@asyncio.coroutine
def first_task():
    for i in itertools.count():
        yield from asyncio.sleep(1)  # настоящий асинх
        print(f'Iteration #{i}')


@asyncio.coroutine
def second_task():
    i = 0
    while True:
        yield from asyncio.sleep(1)  # настоящий асинх
        i = (i + 1) % 5  # каждые 5 секунд

        if i == 0:
            print(time.time())


tasks = [first_task(), second_task()]
loop = asyncio.new_event_loop()

loop.run_until_complete(asyncio.wait(tasks))


In [ ]:
# ЗАПУСКАЙТЕ В ФАЙЛЕ!

import asyncio
import itertools
import time


async def first_task():
    for i in itertools.count():
        await asyncio.sleep(1)  # настоящий асинх
        print(f'Iteration #{i}')


async def second_task():
    i = 0
    while True:
        await asyncio.sleep(1)  # настоящий асинх
        i = (i + 1) % 5  # каждые 5 секунд

        if i == 0:
            print(time.time())


tasks = [first_task(), second_task()]
loop = asyncio.new_event_loop()

loop.run_until_complete(asyncio.wait(tasks))

https://habr.com/ru/post/337420/

In [10]:
import asyncio
import aiohttp

urls = ['http://www.google.com', 'http://www.yandex.ru', 'http://www.python.org']

async def call_url(url):
    print('Starting {}'.format(url))
    response = await aiohttp.get(url)
    data = await response.text()
    print('{}: {} bytes: {}'.format(url, len(data), data))
    return data

futures = [call_url(url) for url in urls]

loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.wait(futures))

C:\Users\Artem\Anaconda3\lib\site-packages\aiohttp\streams.py:606: RuntimeWarning: coroutine 'second_task' was never awaited
  def __aiter__(self) -> AsyncStreamIterator[_T]:
Task exception was never retrieved
future: <Task finished coro=<wait() done, defined at C:\Users\Artem\Anaconda3\lib\asyncio\tasks.py:331> exception=TypeError('An asyncio.Future, a coroutine or an awaitable is required')>
Traceback (most recent call last):
  File "C:\Users\Artem\Anaconda3\lib\asyncio\tasks.py", line 357, in wait
    fs = {ensure_future(f, loop=loop) for f in set(fs)}
  File "C:\Users\Artem\Anaconda3\lib\asyncio\tasks.py", line 357, in <setcomp>
    fs = {ensure_future(f, loop=loop) for f in set(fs)}
  File "C:\Users\Artem\Anaconda3\lib\asyncio\tasks.py", line 588, in ensure_future
    raise TypeError('An asyncio.Future, a coroutine or an awaitable is '
TypeError: An asyncio.Future, a coroutine or an awaitable is required
Task exception was never retrieved
future: <Task finished coro=<wait() done, 

RuntimeError: This event loop is already running

Starting http://www.yandex.ru
Starting http://www.python.org
Starting http://www.google.com
